In [2]:
from bs4 import BeautifulSoup
import requests
import time

regions_list = [
    "https://www.tripadvisor.fr/Attractions-g11038913-Activities-c49-Auvergne_Rhone_Alpes.html",
    "https://www.tripadvisor.fr/Attractions-g11038854-Activities-c49-Bourgogne_Franche_Comte.html",
    "https://www.tripadvisor.fr/Attractions-g187094-Activities-c49-Brittany.html",
    "https://www.tripadvisor.fr/Attractions-g187115-Activities-c49-Centre_Val_de_Loire.html",
    "https://www.tripadvisor.fr/Attractions-g187139-Activities-c49-Corsica.html",
    "https://www.tripadvisor.fr/Attractions-g11038876-Activities-c49-Grand_Est.html",
    "https://www.tripadvisor.fr/Attractions-g11038886-Activities-c49-Hauts_de_France.html",
    "https://www.tripadvisor.fr/Attractions-g187144-Activities-c49-Ile_de_France.html",
    "https://www.tripadvisor.fr/Attractions-g187179-Activities-c49-Normandy.html",
    "https://www.tripadvisor.fr/Attractions-g11038861-Activities-c49-Nouvelle_Aquitaine.html",
    "https://www.tripadvisor.fr/Attractions-g11038881-Activities-c49-Occitanie.html",
    "https://www.tripadvisor.fr/Attractions-g187192-Activities-c49-Pays_de_la_Loire.html",
    "https://www.tripadvisor.fr/Attractions-g187208-Activities-c49-Provence_Alpes_Cote_d_Azur.html"
]


def get_soup(url):
    response = requests.get(url)
    r = response.content
    soup = BeautifulSoup(r, "html.parser")
    return soup
        

def next_page():
    if soup.find(class_="ui_button nav next primary"):
        url = "https://www.tripadvisor.fr" + soup.find(class_="ui_button nav next primary")["href"]
        return url
    else:
        return False


def get_coordinates():    
    scripts = soup.find_all('script')
    for script in scripts:
        if 'window.__WEB_CONTEXT__=' in script.text:
            jsonStr = script.text
            jsonStr = jsonStr.split('window.__WEB_CONTEXT__={pageManifest:')[-1]
    
    try:
        lat = re.findall('"latitude":"(.*?)"', jsonStr)
        lon = re.findall('"longitude":"(.*?)"', jsonStr)

        if len(lat)>0:
            return (lat[0], lon[0])
        else:
            return "Nan"
    except:
        return "Nan"
    
def scrape_page():
    #scraping name of the city
    if soup.find(class_="eQSJNhO6") != None:
        city = soup.find(class_="eQSJNhO6").find("a").text
    else:
        city="NaN"
    city_list.append(city)
    
    if soup.find(class_="ui_header h1") !=None:
        title = soup.find(class_="ui_header h1").text
    else:
        title = "NaN"
    title_list.append(title)
    #scrape ratings
    if soup.find(class_="_1NKYRldB") !=None : 
        rating_temp = soup.find(class_="_1NKYRldB").find_all("span")[0]["class"][1]
    else:
        rating_temp= "NaN"
    rating_list.append(rating_temp)
    # avis
    if soup.find(class_="_1NKYRldB") !=None : 
        avis = soup.find(class_="_1NKYRldB").find_all("span")[1].text
    else : avis = "NaN"
    avis_list.append(avis)
    #labels
    if soup.find(class_="_3RTCF0T0") !=None : 
        labels = soup.find(class_="_3RTCF0T0").text
    else:
        labels = "NaN"
    labels_list.append(labels)
    if soup.find(class_="LjCWTZdN") !=None : 
        adress = soup.find(class_="LjCWTZdN").text
    else:
        adress = "NaN"
    adress_list.append(adress)
    
    coordinates = get_coordinates()
    coo_list.append(coordinates)
    

In [4]:
url_list = []
name_list = []
count = 0
for region in regions_list:
    soup = get_soup(region)
    list_attractions = ["https://www.tripadvisor.fr" + i["href"] for i in soup.find_all(class_="_1QKQOve4")]
    l_names  = [i.find("h2").text for i in soup.find_all(class_="_1QKQOve4")]
    url_list += list_attractions
    name_list += l_names
    print(l_names)
    count +=1
    time.sleep(1)
    next = next_page()
    while next:
        soup = get_soup(next)
        list_attractions = ["https://www.tripadvisor.fr" + i["href"] for i in soup.find_all(class_="_1QKQOve4")]
        l_names  = [i.find("h2").text for i in soup.find_all(class_="_1QKQOve4")]
        url_list += list_attractions
        name_list += l_names
        print(l_names)
        time.sleep(1)
        next = next_page()
        count +=1
        print(count)
print("len url_list : ", len(url_list))
df_url = pd.DataFrame({"name" : name_list, "url" : url_list})
df_url.to_csv("url musées tripadvisor.csv")

["Musée de l'Ecole", "Grand Site Aven d'Orgnac", "Muséum de L'Ardèche", 'Parc du Chateau de Vizille', 'Musée Miniature et Cinéma', "L'Artisan Nougatier", "Le Chaudron d'Or", 'Musée Archéologique', 'Mémorial National de la Prison de Montluc', 'Musee Galo-Romain', 'Mupop', 'Fresque des Lyonnais', 'CNCS - Centre National du Costume de Scène', 'Gallo-Roman Museum', "L'Aventure Michelin", 'Moulin Richard de Bas', 'Musée de Grenoble', 'Les Burons de Salers', 'Institut & Musée Lumière', 'Musée des Beaux-Arts de Lyon', 'Musée Dauphinois', "Musée de la Résistance et de la Déportation de l'Isère", 'Musée de la Résistance', 'Musée des Verts', "Musée Européen de l'Aviation de Chasse", 'Musée de la Révolution francaise', "Centre D'histoire De La Résistance Et De La Déportation", 'Musée de la Coutellerie', 'Château de Grignan', "Musee d'Art et d'Industrie"]
['Musée Opinel', "Musée de l'imprimerie de Lyon", 'Musée des Troupes de Montagne', "Musée de l'Ancien Éveche", 'Musée international de la chauss

10
['Musée du tapis et des arts textiles', 'Le Monde Merveilleux des Lutins', 'Musee Chasal Lento', 'Musée du Vieux Pérouges', 'Maison du Saleve', 'Musee Engrangeons la Memoire', 'Ecomusée -De-Montricher', 'Maison de la Truffe et du Tricastin', 'Musee Maison de la Montagne', 'Gallery Christiane Vallé', 'Ecomusee des Bruneaux', 'Musee du Peigne et de la Plasturgie', 'La Grange aux Pétrolettes', "Maison de l'aventure industrielle  Usine du May", 'Le couvent', "Musee Memoires d'Alpinismes", "Centre d'art Roman", 'Arche des Métiers', 'Musee Barthelemy Thimonnier', 'Maison de Pays - Musee Archeologique du Lac de Paladru', 'Musee surrealiste Francois Boucheix', 'Notre Histoire - Musee de Rumilly', 'Couvent Sainte-Cécile', 'Musée de la cravate et du textile', 'Village de Chessy', 'Musée de la Civilisation Gallo-Romaine', 'La Maison de la Vanoise', 'Musée Georges-Pompidou', 'Musee de la Dentelle', 'Centre écologique Terre Vivante']
11
['Maison Champanhet', 'La Maison des Artisans', "Musée de S

20
['Musée de l’Hôtel-Dieu', 'Musée de la Boissellerie', 'Trésor de Vix - Musée du Pays Châtillonais', 'Musée des Beaux-arts de Dijon', 'Musée de la Résistance et de la Déportation', 'La Grande Saline - Musée du Sel', 'Musée des Maisons Comtoises de Nancray', 'Musée de la Vie Bourguignonne', 'Palais des Ducs', 'La Maison du Visiteur', "Musée de l'Aventure Peugeot", 'Musée du Temps', 'Hameau Duboeuf', "Maison aux Mille Truffes by l'Or des Valois", 'Musée Gustave Courbet', 'Le Cassissium', 'Musée Rude', 'MuséoParc Alésia', 'Musée de la Taillanderie', 'Abbaye Impériale de Baume-les-Messieurs', 'Musée du Jouet', 'Musée Rolin', 'Hôtel-Dieu de Lons-le-Saunier', 'Musee Zervos', 'Musée Nicephore Niépce', "L'Imaginarium", 'Musée Pompon', 'Musée Magnin', 'Dino-Zoo', 'La Maison de La vache qui rit']
['La Maison du Comté', 'Musées de Sens', "Muséum d'Histoire Naturelle", 'Hotel Dieu-Musée Greuze', 'Musée Archéologique', 'Musée de la Mine', "Musée de l'Horlogerie", 'Musée Claude-Nicolas Ledoux', 'M

30
['Musée Mémoires 39-45', 'Univers du Poète Ferrailleur', "Manoir de l'Automobile", 'Naïa Museum', 'Le Fonds Hélène et Édouard Leclerc pour la Culture ', 'Haliotika - La Cité de la Pêche', 'Musée de la Compagnie des Indes', 'Village de Poul-Fetan', 'Musée Vivant des Vieux Métiers', 'Site de Meneham', 'Musée des Phares et Balises', 'Musée Départemental Breton', 'Musée Des Beaux-Arts De Quimper', 'Musee de la Peche', 'Port-Musée', 'Base de sous-marins de Keroman', 'Cité des Télécoms', 'Maison de Sarah Bernhardt', 'Musée de Pont-Aven', 'Cathédraloscope', 'Les Champs Libres', 'Musée national de la Marine', 'Sous-Marin Flore', 'Cite de la Voile Eric Tabarly', 'Ancienne Abbaye de Landévennec', 'Musée Citadelle Vauban', 'Insectarium de Lizio', 'Cidrerie Kerne', 'Musée des Beaux-Arts de Rennes', 'Musée National de la Marine']
['Marinarium', 'Ecomusée du pays de Rennes', 'Musée de Préhistoire', 'Maison a Pondalez', 'Etoile du Roy', 'Usine marémotrice EDF de la Rance', "Ecomusée d'Ouessant : M

41
['Musee de Sologne', 'Musée des Arts Décoratifs', 'Canopee', 'Maison du Cerf', 'Le Moulin des Aigremonts', 'Chapelle Saint Eman', 'Musée du Safran', 'Auto Sport Museum', 'Musée de la Sorcellerie', 'Musée de la Guerre de 1870', "Musée d'Art Urbain - Street Art Parc", 'Musée Emile Chénon', 'La Maison des Traditions de Chassignolles', 'Musee du Cirque', "Musee d'Art religieux", 'Galerie le Garage', 'Domaine Les Lutinières', 'Musée Saint Martin', 'Soline du Puy', 'Musee ateliers des pionniers', 'Musée Du Collège Royal Et Militaire De Thiron-Gardais', 'Musée du Matériel', "Cite de l'Or - Pyramide des Metiers d'Art", "La Tuilerie, Centre d'Interpretation de l'Architecture et du Patrimoine", 'Galerie Magi', 'Artsenal', 'Galerie Cravantaise', 'Galerie Garnier Delaporte', "L'Esprit du Parfum", 'Musée des Fours Banaux']
42
['Chateau Moncontour', 'Montoire sur Loire Museum', 'Musée des Mariniers', 'Musée du Train', 'Musée de la Viellle Alliance Franco Ecossaise', 'Prieure Saint Vincent', "Musé

...

94
['Musée Hèbre', 'Cap Sciences', 'Espace Rebeyrolle', 'Ecomusée du Cognac', "Chateau d'Oiron", 'Centre National Jean Moulin', 'M\xadusée Despiau-Wlérick\xad\xad\xad', 'Château Chasse-Spleen', "Musee Landes d'Antan", 'Château Lafite', 'Maison du Lillet', '\u200b\ufeffAt\u200belier-Musee des Cartons de Tapisserie', 'Prison des eveques', 'MUSEE DE LA CHALOSSE', 'La Vieille Eglise Saint-Vincent Mérignac', 'Maison de la Boucherie', 'Notre Dame des Cyclistes', "Musee de l'Homme de neandertal", 'Musee du Liege', "Conservatoire des Métiers d'Autrefois", 'Les Tresors de Lisette', 'Musée de la Voiture à Cheval', 'Villa gallo-romaine', 'Musée Bernadotte', 'La Maison des Douanes', 'Galerie Valerie Peter', 'Musée de la Ville (du Vin et de la Batellerie)', 'La Gare Robert Doisneau', "Musée Jeanne d'Albret", 'Musée des Arts Decoratifs']
95
["Le Trésor de la Cathédrale d'Angoulême", 'Musee des Cordeliers', 'Une Nouvelle Empreinte', "Musée d'Art et d'Histoire", "Musee d'art Populaire", 'Papèterie de 

104
['Ecomusee du Libournais', 'Art Home Deco', 'Mémorial De La Résistance', 'Maison du Parc national des Pyrénées de Etsaut', 'Morpho Bleu', 'en apARTe', "Atelier D'art Julien", "Atelier d'Art Julien", "Galerie d'Art du Passage", "Cyril More Tourneur D'art Sur Bois", 'Atelier Bike and Roll', 'Atelier Brigitte Jacques', 'Musée du Landaus', 'Moulin Laxia', 'Francis Jammes', 'Espace Fer et Forges', 'Musée d’Art religieux (couvent des célestins)', 'Exposition Espace Noel-J', 'Musee vieux metiers']
105
['Musée du Patrimoine Agricole et Automobile de Salviac', 'Musée 1900', 'Un Siècle de Roulottes', 'Musée des Jouets', "Ferme Caussenarde d'Autréfois", 'Grotte du Pech Merle', 'le Moulin a Papier de Brousses', 'Musee du Desert', 'Météosite du Mont Aigoual', 'Musée Dom Robert et de la tapisserie du XXe siècle', 'Dynamiterie de Paulilles', 'Les Trains du Colorado', 'Ailes Anciennes Toulouse', 'Espace Georges  Brassens', 'Le Petit Lourdes', 'Musée Fabre', 'Musée-Mine Départemental', 'Musée Fenai

115
['Musee de Javols', 'Espace 213 – Auvill’Art', 'Maison de la Chataigne', 'Musee Marcel Lenoir', 'Le Musee du Compagnonnage', 'Musée de la Vie Paysanne en Haut-Languedoc', 'Musée du Paysan Gascon', "Les Chais D'Armagnac", 'Centre de Ceramique de Giroussens', "Centre d'Interpretation des Megalithes", 'Bull Cultures Museum - Musée des cultures taurines', 'Musee des Beaux-Arts  Salies', 'Espace Rabelais', 'Moulin des Eveques', 'Carla Bayle La Cite des Arts', "Centre d'art Le LAIT", 'Espace Molière', 'Galerie des Hospices', 'Église Saint-Benoît', 'La Tour de Guet', 'Galerie Nadine Granier', 'Musee Calbet', "Musee de l'Abbaye", 'Observatoire de Jolimont', 'Galerie du Causse', "Maison de l'Eau et de la Méditerranée", 'Musée de la Déportation et de la Résistance', 'Musée des Métiers de la Pierre et de la vie Rurale', "L'Epopée Chapelière", 'Musee des Poupees et des Nounours']
116
['Musée Municipal Urbain Cabrol', "Centre D'art Contemporain Walter Benjamin", 'Les Meubles Modestes', 'Atelier

125
["De la Vague a l'Art", "Terre d'estuaire", 'Musée Dobrée', 'La Cité des Oiseaux', "Musée Aéronautique de la Presqu'île Côte d'Amour", 'La Maison de la Pêche', 'Bains Douches Municipaux', 'Musée et Site archéologique Jublains', "Carnuta - L'Homme et la Forêt", 'Musée Jules-Desbois', 'Moulins à Vent Jumeaux du Terrier Marteau', "Musée de la Vigne et du Vin d'Anjou", 'Musee Vert', 'Musee des Sciences de Laval', 'L’Helice Terrestre de l’Orbiere', 'Le Musée des Métiers', 'Musée Jules Verne de Nantes', 'La Feerie des Santons', 'Musée du Pays de Retz', 'Musee du Pain de la Mie Caline', 'Centre Beautour', 'Musée Joseph-Denais', "La Maison de l'Artisanat d'Art", "Musée de l'Ardoise", 'Musée de La Roche-sur-Yon', "Musée Régional de l'Air - Espace Air Passion", 'Musée de la Résistance de Châteaubriant', "Musée d'Histoire Urbaine", 'Musée ou Logis Pincé', 'Musée Bernard Chardon']
126
['Musée du Bon Pasteur', 'Musée Des Traditions Populaires', 'Kartonik Galerie Atelier Boutique', 'Rotonde Ferr

136
['Mazarin', 'Musée du Vieux Toulon', 'Musée du Verre et du Vitrail', 'Fondation Blachere', 'Musée de la Lavande', 'Eglise Saint-Christophe', 'Musee Frederic Mistral', 'Hotel de Sade', 'Musee de Cucuron', 'Fondation Carzou', 'Le Château de Villeneuve-Loubet', 'Musée Renoir/Les Collettes', "Centre d'Art des Pénitents Noirs", 'Ecomusée du Cheminot Veynois', "Musee D'Art Sacre", 'Chapelle Bellini', 'La Maison du Portal', 'Harmas de Jean-Henri Fabre', 'Villa Méditerrannée', 'Villa Carmignac', 'Musée de la Tour Carrée', 'Musée de Terra Amata', 'Musée Muséum départemental des Hautes-Alpes', 'Musee de Region Auguste Chabaud', 'retouR De Voyage', 'Musée des Chantiers Naval', 'Château de la Clapière', "Musee d'Arts Africains, Oceaniens et Amerindiens (MAAOA)", 'Musée de la Mine', 'Château de Gordes']
137
['Galerie Gabel', 'Chateau-Musee Tourrette-Levens', 'Maison de la Biodiversite', "Musée Aéronautique d'Orange", 'Musée de la distillerie', 'The Gallery', 'La Maison des Papillons', 'Ecomusee

In [10]:
#initialize data storage
city_list = []
rating_list = []
avis_list = []
labels_list = []
adress_list = []
coordinates_list = []
title_list = []
coo_list= []


nb = 0
for url in url_list:
    soup = get_soup(url)
    scrape_page()
    nb +=1
    time.sleep(1)
    if nb%50 == 0:
        print(nb)
    
df_infos = pd.DataFrame({"city" : city_list, "title" : title_list, "rating" : rating_list, "avis" : avis_list, "labels" : labels_list, "adress" : adress_list, "coordinates" : coo_list})
df_infos.to_csv("liste musées 3.csv")

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200


In [18]:
df_infos[(df_infos["labels"] != "Galeries d'art") & (df_infos['adress'] == "NaN")]

,city,title,rating,avis,labels,adress,coordinates
101,choses à voir/à faire à Saint-Ours,La Ruche des Puys,bubble_50,43 avis,Musées spécialisés,NaN,Nan
161,choses à voir/à faire à Brassac-les-Mines,Musee de la Mine,bubble_45,26 avis,Musées spécialisés,NaN,Nan
186,choses à voir/à faire à Hieres-sur-Amby,Musee-Maison du Patrimoine,bubble_50,1 avis,Musées spécialisés,NaN,Nan
190,choses à voir/à faire à Laqueuille,De Bleu et de Sang,bubble_50,1 avis,Musées spécialisés,NaN,Nan
239,choses à voir/à faire à Le Poet-Laval,Musée du Protestantisme Dauphinois,bubble_45,29 avis,"Musées spécialisés, Musées d'histoire",NaN,Nan
...,...,...,...,...,...,...,...
4111,choses à voir/à faire à Thoard,Le Petit Musée des Cuivres,bubble_50,1 avis,Musées spécialisés,NaN,Nan
4125,choses à voir/à faire à Saint-Bonnet en Champsaur,Ecomusée du Noyer,bubble_40,1 avis,Musées spécialisés,NaN,Nan
4138,choses à voir/à faire à Salon-de-Provence,Musée de Salon et de la Crau,bubble_25,2 avis,Musées d'art,NaN,Nan
4144,choses à voir/à faire à Les Mées,Musée de l'Olivier,bubble_20,2 avis,Musées spécialisés,NaN,Nan


# cleaning data


In [346]:
df_infos2['avis'] = df_infos2['avis'].str.replace(" avis","")
df_infos2['avis'] = df_infos2['avis'].str.replace("NaN","0") 
df_infos2['avis'] = df_infos2['avis'].str.replace(" ","")
df_infos2['avis'] = df_infos2['avis'].str.replace("\xa0","")
df_infos2['avis'] = pd.to_numeric(df_infos2['avis'])

df_infos2['city'] = df_infos2['adress'].str.extract("\d{5}(.*)F")
df_infos2['city'] = df_infos2['city'].str.replace(",","")

df_infos2["dep"] = df_infos2["adress"].str.extract("(\d{5})")
df_infos2["adress"] = df_infos2["adress"].str.replace(" France", "")

In [249]:
df_clean = df_infos2[(df_infos2["labels"] != "Galeries d'art")].drop_duplicates(subset=["title","city"])
df_clean["dep2"] = df_clean["dep"].apply(lambda x: str(x)[:2])
dep = pd.read_html("https://www.regions-et-departements.fr/departements-francais")[0]
dep = dep.replace("2B","20").replace("2A","20")
df_clean = df_clean.merge(dep, how="left", left_on="dep2", right_on="NUMÉROdu département").drop(["dep2","NUMÉROdu département"], axis=1)
df_clean["rating"] = df_clean["rating"].astype("float64")
df_clean["adress"] = df_clean["adress"].fillna("")
df_clean.to_csv("liste musée clean.csv")
df_clean

NameError: name 'df_infos2' is not defined

## On continue

In [153]:
nb_par_dep = df_clean["NOMdu département"].value_counts().sort_index()
avis_cum = df_clean.groupby("NOMdu département")['avis'].sum().sort_index()
avis_cum

NOMdu département
Ain                           576
Aisne                         920
Allier                       1420
Alpes-Maritimes             22762
Alpes-de-Haute-Provence      1496
Ardennes                      399
Ardèche                      3708
Ariège                       1111
Aube                         1206
Aude                         2842
Aveyron                      2672
Bas-Rhin                     8455
Bouches-du-Rhône            28579
Calvados                    29747
Cantal                        532
Charente                      975
Charente-Maritime            6682
Cher                          838
Corrèze                       724
Corse-du-Sud                 3694
Creuse                        419
Côte d'Or                     251
Côte-d'Or                   11139
Côte-d’Or                       9
Côtes d'Armor                1090
Deux-Sèvres                   618
Dordogne                     8282
Doubs                        2916
Drôme                        6

In [287]:
#Nettoyage pour importation dans qgis
df_qgis = df_clean.copy()
def clean_tup(x):
    x = x.lstrip("(")
    x = x.rstrip(")")
    x = x.split(",")
    return x
df_qgis["coordinates"] = df_clean["coordinates"].apply(clean_tup)

temp = df_qgis["coordinates"].apply(pd.Series)
temp.columns = ["latitude","longitude"]
df_qgis = pd.concat([df_qgis,temp],axis=1)
df_qgis["latitude"] = df_qgis["latitude"].str.replace("'","").replace('"',"").astype("float64")
df_qgis["longitude"] = df_qgis["longitude"].str.replace("'","").replace('"',"").astype("float64")
df_qgis.drop_duplicates(subset=["title","avis","rating"])
df_qgis.to_csv('qgis musées.csv')